Collaborative filtering: memory approach ¶
Collaborative filtering has quickly become one of the most used techniques in the construction of recommendation systems.

The site Amazon, whose initial activity only concerned the sale of books online, popularized this method with its service "People who liked this book also liked this book".

Collaborative filtering consists of providing recommendations by exclusively exploiting past interactions in users and products, by grouping and identifying groups of users or products whose interactions are similar.

More precisely, collaborative filtering uses a rating matrix whose rows correspond to users and columns to elements. Thus, the cell (u, i) of the matrix corresponds either to the score given by the user u to the product i or to an empty box if the user has not had any interaction with the product.

This score associated with a user and a product can be collected in two ways:

Explicit: the user assigns ratings to the products tested or explicitly indicates their preferences (likes/dislikes)
Implicit: the score is calculated based on the user's interactions with this product (seen, put in the basket, purchased, duration on the page, etc.)
The goal of collaborative filtering is to predict the score that a user would give to a product with which they have not yet tested. Thus, the better the prediction, the more relevant the product will be to recommend to the user.

The collaborative filtering technique can be implemented using two approaches that we will explore: the memory approach and the model approach.

In [ ]:
# dataset link for this part:
# https://cseweb.ucsd.edu/~jmcauley/datasets/goodreads.html#datasets
# in Book Reviews section and then only english reviews 

In [ ]:

import pandas as pd

# Data import
df = pd.read_csv('ratings_flrd_downloaded.csv')

# Display of the first 5 lines
df.head(10)

We will again work with Goodreads datasets but, unlike content-based filtering, we do not use the characteristics of the books but simply the ratings that users have assigned to them. For this, we mainly focus on the database containing mainly the comments that users have given to the books.

This time the dataset to be cleaned contains 15 million comments in different languages, we have 11 columns: 'user_id', 'book_id', 'review_id', 'rating', 'review_text', 'date_added', 'date_updated' ', 'read_at', 'started_at', 'n_votes' and 'n_comments'.

We see that this dataset does not contain the titles of the books but it contains 'book_id' which will allow us to make a merge witht he previous dataset to display the titles. As the comments are in all languages, we do pre-processing work to keep only the comments in English.

Once the merging and filtering are done, the variables that will interest us are: id_user, title and rating. Finally, we will filter to keep only users who have rated at least 10 books and books that have been rated at least twice. This allows us to have a dataset that is easier to manipulate.

# Viz
We can note that the first lines relate to the same book, however the user identifiers are different: the books have more than 2 ratings but users can only rate a book once. Furthermore, we only have users who have rated at least 10 different books.

Now, let's display some graphs to see the composition of our database

In [ ]:
#  In a bar chart, display the distribution of ratings.
import seaborn as sns 
import matplotlib.pyplot as plt

sns.countplot(x="rating", data=df, palette="viridis")
plt.title("Notes distribution", fontsize=14)
plt.show();

In a bar chart, display the 10 highest rated books
   Please note, the same book can appear several times if it has been rated by more than one user, its title remains unique.


In [ ]:
# Group by book title and calculate the quantity of ratings
aggregated_data = df.groupby('title')['rating'].count().reset_index()

# Sorting the aggregated DataFrame by rating in descending order
sorted_aggregated_data = aggregated_data.sort_values(by='rating', ascending=False)

# Selection of the first 10 best-rated books
top_rated_books = sorted_aggregated_data.head(10)

# Graph display
sns.barplot(y='title', x='rating', data=top_rated_books, orient = 'h')
plt.xlabel('Amount of reviews')
plt.ylabel('Title')
plt.title(f'Top 10 books');

In [ ]:
#best rated books
# Group by book title and calculate average rating
aggregated_data = df.groupby('title')['rating'].mean().reset_index()

# Sorting the aggregated DataFrame by rating in descending order
sorted_aggregated_data = aggregated_data.sort_values(by='rating', ascending=False)

# Selection of the first 10 best-rated books
best_rated_books = sorted_aggregated_data.head(10)

# Graph display
sns.barplot(y='title', x='rating', data=best_rated_books, orient = 'h')
plt.title(f'Top 10 best rated books')
plt.xlabel("Average note");

Note that the last graph does not take into account the fact that there are books which are rated more times than others: we can have a book which is rated only once and has a rating of 5 while we may have books that have been rated 10 times and have a 4.9 rating. To take the popularity of a book into account, we use the Bayesian average.

Bayesian averaging is a statistical method for calculating a more robust estimate of the average rating of a set of books, particularly when there are books with a small number of ratings. This approach is particularly useful when we want to evaluate the average quality of the most highly rated books.  𝐵𝑎𝑦𝑒𝑠𝑖𝑎𝑛_𝑎𝑣𝑒𝑟𝑎𝑔𝑒=𝐶×𝑀+𝑆𝐶+𝑛,
  where

𝑀
  = raw average book ratings.
𝑛
  = the total amount of notes.
𝑆
  = the sum of the notes.
𝐶
  = average of the quantity of notes.
(e) Calculate the amount of ratings per book as well as the average rating per book. We will store this information in a variable named book_stats.

(f) With this information, calculate C and M, then create a function that calculates the Bayesian mean. Then, add a column to your book_stats variable with the calculations of the Bayesian average per book.

(g) In a bar chart, display the highest rated books based on the Bayesian average.

In [ ]:
# We group the data by title and calculate the number of ratings ('count') and the average rating ('mean') for each book.
book_stats = df.groupby('title')['rating'].agg(['count', 'mean']).reset_index()

# We calculate the average number of reviews for all books.
C = book_stats['count'].mean()

# We calculate the average of the average scores for all the books.
M = book_stats['mean'].mean()

# We define the function 'bayesian_avg' which calculates the Bayesian rating for each book using the values ​​of C and M calculated previously.
def bayesian_avg(df):
    return (C * M + df.sum()) / (C + df.count())

# We calculate the Bayesian rating for each book using the 'bayesian_avg' function.
bayesian_avg_ratings = df.groupby('title')['rating'].agg(bayesian_avg).reset_index()

# We rename the columns of the DataFrame 'bayesian_avg_ratings' to make them more explicit.
bayesian_avg_ratings.columns = ['title', 'bayesian_avg']

# We merge 'book_stats' with the Bayesian averages using the title as a key and we sort by Bayesian average in descending order.
book_stats = book_stats.merge(bayesian_avg_ratings, on='title').sort_values('bayesian_avg', ascending=False)

## Selection of the first 10 best-rated books
best_rated_books = book_stats[['title', 'bayesian_avg']].head(10)

# Graph display
sns.barplot(y='title', x='bayesian_avg', data=best_rated_books, orient = 'h')
plt.title(f'Top 10 best rated books : bayesian average')
plt.xlabel("Average note")

The memory approach: definition and explanations ¶
The memory approach is based on the correlation between users' "past" behaviors

To do this, it is based on a rating matrix where the rows represent the users and the columns represent the content, here the books.

Thus, cell (u, i) of the rating matrix then corresponds to the rating given by user u to content i.

These interactions can be analyzed in two ways:

Either the analysis is based on users (user-based filtering)
Either the analysis is based on the products, (item-based filtering)
Let’s illustrate these two approaches:

2.1 User-based filtering
This approach is based on the idea that a user is likely to have the same preferences as a user with the same rating behavior as them.

The vector associated with Rafaela (in yellow) is very correlated to that of Benoit (in dark blue).

Rafaela and Benoit have the same tastes in terms of books. Rafaela really liked Harry Potter 1 and Benoit hasn't read it. We could then predict that Benoit will like this book and recommend it to him.


2.2 Item-based filtering
This approach is based on the idea that two films which have received the same ratings from the same user are likely to be similar.

The vector associated with the Chronicles of Narnia (in purple) and that associated with Harry Potter 1 (in light blue) are very correlated.

Both books received the same ratings from the same user. We can therefore predict that the two books are similar. Benoit really enjoyed reading Narnia, so we can predict that he will love Harry Potter just as much and recommend it to him.


 Attention To bring two products together, it is not necessary that the two products have received the same ratings from all users but simply that their rating vectors are correlated.
  Here, we could then interpret the connection between Narnia and Harry Potter by their fantastic character. A person who doesn't like fantasy will give these two books a low rating and vice versa.


# 3. Implementation 
3.1 Creation of the rating matrix
   This step is common to both approaches: user-based and item-based filtering
We will build the ratings matrix where each row represents the ratings given by a user and each column the ratings assigned to content.

Thus, the cell (u, i) of the model matrix then corresponds to the rating given by the user u to the content i.

(h) Create two variables n_users and n_books containing the number of users and the number of books, respectively.

In [ ]:
n_users = df['user_id'].nunique()

n_books = df['title'].nunique()

print("Number of users: ", n_users)

print("Number of books: ", n_books)

(i) Create the ratings matrix associated with the DataFrame by taking the user identifiers as an index and the book titles as columns, storing the matrix in a variable named mat_ratings.
   The pivot_table method allows you to create a matrix from a DataFrame.

In [ ]:

# Use the 'pivot_table' method to create the ratings matrix.
# 'columns = 'title'' specifies that the columns of the matrix correspond to the titles of the books.
# 'index = 'user_id'' specifies that the rows correspond to user IDs.
# 'values ​​= 'rating'' specifies that the values ​​in the matrix are the ratings given by users.
mat_ratings = df.pivot_table(columns='title', index='user_id', values='rating')

# Show the first 10 rows of the matrix.
mat_ratings.head(10)

In the next steps, our goal is to substitute missing values ​​in this matrix with zeros. However, it is essential not to distort our rating system, which also starts from 0.

(j) Increase by 1 all the ratings given by users.
   Our rating system will now range from 1 to 6 instead of 0 to 5.

In [ ]:
mat_ratings = mat_ratings +1

mat_ratings

 Replace missing values ​​with zeros.

In [ ]:
mat_ratings.fillna(0, inplace=True)

mat_ratings

Note that the rating matrix is very large and contains many missing values. This can be heavy in terms of storage and slow down calculations.

Indeed, most of the entries in this matrix are not observed, because a given user only reads and rates a small number of books compared to the total number of books in the dataset.

These matrices containing a large number of zero values ​​are called sparse matrices or sparse matrix in English.

When working with sparse matrices, it is then interesting to use a CSR (Compressed Sparse Row) format.

The CSR format only stores non-zero elements of the matrix with their column indices and row pointers. This results in significantly less memory usage.

(l) Store in a sparse_ratings variable the mat_ratings matrix in CSR format. We will also store user IDs and book titles in variables named user_ids and titles, respectively. This will be useful to us later.

 The csr_matrix method of scipy.sparse allows you to transform a matrix into CSR format

In [ ]:
from scipy.sparse import csr_matrix

# Convert the 'mat_ratings' rating matrix to a 'sparse_ratings' sparse matrix.
sparse_ratings = csr_matrix(mat_ratings)

# Extract user IDs and book titles from the ratings matrix.
user_ids = mat_ratings.index.tolist()  
titles = mat_ratings.columns.tolist()  

# Show sparse matrix 'sparse_ratings'.
print(sparse_ratings)

The CSR matrix tells us in parentheses the rank, the column and then the value other than zero on the matrix. In our case we have (user, book) note given.

2.2 User-based filtering
The similarity matrix

In this approach, recommendations for a user are made by exploring the preferences of the most similar users. We must therefore be able to assess the extent to which two individuals are similar.

For this we use the cosine similarity between the users.

Reminder: The cosine similarity  𝑠(𝑥,𝑦)
  between two vectors  𝑥
  and  𝑦
  is:

𝑠(𝑥,𝑦)=cos(𝑥,𝑦)=∑𝑑𝑖=1𝑥𝑖𝑦𝑖‖𝑥‖‖𝑦‖where 𝑑 is the dimension of the vectors and ‖𝑥‖=∑𝑑𝑖=1𝑥2𝑖⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯√
 
(a) Create the function sim_cos() taking two vectors as arguments:  𝑥
  and  𝑦
  and returning the cosine similarity between these two vectors. If either vector is zero, the function will return 0.

In [ ]:
import numpy as np

# Definition of a function 'sim_cos' to calculate the cosine similarity between two vectors 'x' and 'y'.
def sim_cos(x, y):
    # Calculation of the scalar product between the vectors 'x' and 'y'.
    dot_product = np.dot(x, y)
    
    # Calculation of Euclidean norms of 'x' and 'y'.
    norm_x = np.sqrt(np.sum(x ** 2))
    norm_y = np.sqrt(np.sum(y ** 2))
    
    # Checking if one of the standards is zero to avoid division by zero.
    if norm_x == 0 or norm_y == 0:
        return 0
    
    # Calculation of cosine similarity using the formula.
    similarity = dot_product / (norm_x * norm_y)
    return similarity

b) Store in the vectors pref_1 and pref_2 the user ratings 00b05fbae777bb4325cffc46313e2179 and 00ce07379fb4a962964dcfde4e146a84. Then, from mat_ratings, test the sim_cos() function on these last two.

In [ ]:
# Insert your answer here

pref_1 = mat_ratings.loc["00b05fbae777bb4325cffc46313e2179", :].values

pref_2 = mat_ratings.loc["00ce07379fb4a962964dcfde4e146a84", :].values

similarity = sim_cos(pref_1, pref_2)


print("The similarity between the two users is ", similarity)

  The cosine_similarity function from the sklearn.metrics.pairwise library allows you to calculate cosine similarity. It has the advantage of being able to be applied to a CSR matrix.
(c) Create the matrix user_similarity using the cosine_similarity function, it must be of dimension  (2023×2023)
 . Then convert the matrix to DataFrame. To ensure that we have the right users corresponding to each similarity, we will use the users_ids variable defined previously for the index and columns parameters. As a reminder, it contains the user identifiers in the order of the rating matrix.

In [ ]:
import sklearn.metrics.pairwise as dist

# Using the 'cosine_similarity' function of the 'dist' module to calculate the cosine similarity between users.
user_similarity = dist.cosine_similarity(sparse_ratings)

# Creation of a pandas DataFrame from the similarity matrix between users.
# The indexes and columns of the DataFrame are the user identifiers.
user_similarity = pd.DataFrame(user_similarity, index=user_ids, columns=user_ids)